# 10 神经网络各层 FLOPs 详细推导

> 注：统一假设一次乘法 = 1 FLOP，一次加法 = 1 FLOP  
> 一个乘-加（MAC）= 2 FLOPs（业界通用标准）

---

## 10.1 全连接层（Linear / Dense）

输入向量 $\mathbf{x} \in \mathbb{R}^{I}$，权重矩阵 $\mathbf{W} \in \mathbb{R}^{O \times I}$，输出 $\mathbf{y} = \mathbf{W}\mathbf{x} + \mathbf{b}$

- 计算第 $j$ 个输出分量 $y_j$ 需要：
  - $I$ 次乘法：$w_{j1}x_1 + w_{j2}x_2 + \cdots + w_{jI}x_I$
  - $I-1$ 次加法（严格）≈ $I$ 次加法（大模型中忽略 -1）
- 总共 $O$ 个输出分量
- 所以一次前向传播（单样本）：
  $$
  \begin{aligned}
  \text{乘法次数} &= O \times I \\
  \text{加法次数} &= O \times I \\
  \text{FLOPs} &= 2OI
  \end{aligned}
  $$
- 若 batch size = $B$，则输入变成 $B \times I$，所有计算乘以 $B$：
  $$
  \boxed{\text{FLOPs}_{\text{Linear}} = 2 B I O}
  $$

---

## 10.2 普通二维卷积层（Conv2D

输入特征图：$B \times C_{\text{in}} \times H_{\text{in}} \times W_{\text{in}}$  
卷积核：$C_{\text{out}} \times C_{\text{in}} \times K_h \times K_w$  
输出特征图：$B \times C_{\text{out}} \times H_{\text{out}} \times W_{\text{out}}$

我们只算一个输出像素 $(b, c_{\text{out}}, i, j)$ 需要多少运算：

1. 要产生这个像素，需要把卷积核在输入上滑动到对应位置
2. 卷积核有 $C_{\text{in}} \times K_h \times K_w$ 个权重
3. 每个权重与输入对应位置相乘 → $C_{\text{in}} K_h K_w$ 次乘法
4. 把这些乘积全部加起来 → $C_{\text{in}} K_h K_w - 1 \approx C_{\text{in}} K_h K_w$ 次加法
5. 所以一个输出像素需要约 $2 C_{\text{in}} K_h K_w$ FLOPs

现在统计总共有多少个输出像素：
- batch 有 $B$ 个
- 输出通道有 $C_{\text{out}}$ 个
- 输出空间位置有 $H_{\text{out}} \times W_{\text{out}}$ 个

因此总 FLOPs：
$$
\begin{aligned}
\text{FLOPs}_{\text{Conv2D}} 
&= B \times C_{\text{out}} \times H_{\text{out}} \times W_{\text{out}} \times 2 C_{\text{in}} K_h K_w \\
&= \boxed{2 B C_{\text{in}} C_{\text{out}} K_h K_w H_{\text{out}} W_{\text{out}}}
\end{aligned}
$$

---

## 10.3. 深度可分离卷积（Depthwise Separable Convolution）

分成两步：Depthwise + Pointwise

#### (1) Depthwise 卷积（每个输入通道单独卷积）
- 卷积核：$C_{\text{in}} \times 1 \times K_h \times K_w$（每个输入通道一个 2D 核）
- 一个输出像素（单通道）需要 $K_h K_w$ 次乘法 + $K_h K_w$ 次加法
- 总输出像素数：$B \times C_{\text{in}} \times H_{\text{out}} \times W_{\text{out}}$
- 所以：
  $$
  \text{FLOPs}_{\text{depthwise}} = 2 B C_{\text{in}} K_h K_w H_{\text{out}} W_{\text{out}}
  $$

#### (2) Pointwise 卷积（1×1 普通卷积）
- 卷积核：$C_{\text{out}} \times C_{\text{in}} \times 1 \times 1$
- 相当于全连接层在通道维度上
- FLOPs：
  $$
  \text{FLOPs}_{\text{pointwise}} = 2 B C_{\text{in}} C_{\text{out}} H_{\text{out}} W_{\text{out}}
  $$

总计：
$$
\boxed{\text{FLOPs}_{\text{Separable}} = 2 B C_{\text{in}} K_h K_w H_{\text{out}} W_{\text{out}} + 2 B C_{\text{in}} C_{\text{out}} H_{\text{out}} W_{\text{out}}}
$$

---

## 10.4. Transformer Self-Attention（多头）

输入：$X \in \mathbb{R}^{B \times L \times d_{\text{model}}}$

### 步骤 1：线性投影生成 Q、K、V（3 次线性变换）
每一次投影：$X W_q \ (d_{\text{model}} \times d_{\text{model}})$  
FLOPs = $2 B L d_{\text{model}}^2$  
三组 QKV 总计：
$$
3 \times 2 B L d_{\text{model}}^2 = 6 B L d_{\text{model}}^2
$$

### 步骤 2：注意力分数 $A = \text{softmax}(Q K^T / \sqrt{d_k})$
$Q K^T$：$(B L d) \times (d \times L) \to B \times L \times L$  
每次计算一个 $q_i \cdot k_j$ 需要 $d_{\text{model}}$ 次乘加 → $2 d_{\text{model}}$ FLOPs  
总共 $L \times L$ 个点 → 单头：
$$
2 B L^2 d_{\text{model}}
$$
（多头并行计算，不额外乘 h）

### 步骤 3：注意力输出 $O = A V$
$A (B L L) \times V (B L d)$  
每个输出位置需要 $L$ 次乘加 → $2 L d_{\text{model}}$ FLOPs  
总共 $B L$ 个位置：
$$
2 B L^2 d_{\text{model}}
$$

### 步骤 4：输出投影（又一个线性层）
$$
2 B L d_{\text{model}}^2
$$

### Attention 总计：
$$
\boxed{\text{FLOPs}_{\text{MultiHead-Attention}} = 4 B L^2 d_{\text{model}} + 8 B L d_{\text{model}}^2}
$$
（其中 $4BL^2d$ 来自 QK^T 和 AV，$8BLd^2$ 来自四个线性层）

很多论文会把四个线性层写成 $2BLd^2$（因为多头内部共享计算），但最严格是上面这个。

### 步骤 5：Feed-Forward Network（FFN）
通常是两层：
$$
X \to X W_1 + b_1 \to \text{GeLU} \to X W_2 + b_2
$$
$W_1 \in \mathbb{R}^{d \times 4d}$，$W_2 \in \mathbb{R}^{4d \times d}$

第一层：$2 B L d \cdot 4d = 8 B L d^2$  
第二层：$2 B L \cdot 4d \cdot d = 8 B L d^2$  
总计：
$$
\boxed{\text{FLOPs}_{\text{FFN}} = 16 B L d_{\text{model}}^2}
$$
（原始 Transformer 是 8d → 8×2=16）

但 GPT/BERT 系列常用近似系数 4，最终常写成 8BLd²（隐藏层 4d → 回来 d）

### 单层 Transformer 总 FLOPs：
$$
\boxed{
\begin{aligned}
\text{FLOPs}_{\text{one layer}} &= \text{Attention} + \text{FFN} \\
&= 4 B L^2 d + 8 B L d^2 + 16 B L d^2 \\
&= 4 B L^2 d_{\text{model}} + 24 B L d_{\text{model}}^2
\end{aligned}
}
$$

业界最常用近似（OpenAI、PaLM、LLaMA 论文）：
$$
\boxed{\text{FLOPs}_{\text{per token}} \approx 12 d_{\text{model}}^2 \quad \Rightarrow \quad \text{FLOPs}_{\text{total}} \approx 12 B N L d_{\text{model}}^2}
$$
（把 $L^2$ 项也近似成 $L d$，在长序列时不成立，但在训练时 $L$ 通常不太大，所以被接受）

---

## 10.5 总结表格
### 10.5.1 常见Ops FLOPs

| 层类型                   | 详细 FLOPs 公式（含 batch $B$）                                                                                  |
|--------------------------|-----------------------------------------------------------------------------------------------------------------|
| 全连接层                 | $2 B I O$                                                                                                       |
| 普通 Conv2D              | $2 B C_{\text{in}} C_{\text{out}} K_h K_w H_{\text{out}} W_{\text{out}}$                                        |
| Depthwise Conv           | $2 B C_{\text{in}} K_h K_w H_{\text{out}} W_{\text{out}}$                                                        |
| Pointwise (1×1) Conv     | $2 B C_{\text{in}} C_{\text{out}} H_{\text{out}} W_{\text{out}}$                                                 |
| Attention (QKV + Out)    | $4 B L^2 d + 8 B L d^2$                                                                                         |
| FFN (4d 中间)            | $16 B L d^2$                                                                                                    |
| 单层 Transformer         | $4 B L^2 d + 24 B L d^2$                                                                                         |
| 完整模型近似（最常用）   | $12 B N L d^2$（$N$ 为层数， 工程近似）


Transformer 
- 当 L ≥ D 时，L² 项 dominate

- 当 D ≥ L 时，MLP dominate


## 10.5.2 常见大模型 FLOPs

| 模型                    | FLOPs (单输入)            | 备注         |
| --------------------- | ---------------------- | ---------- |
| ResNet-18             | ~1.8 GFLOPs            | 224×224    |
| ResNet-50             | ~4 GFLOPs              | 224×224    |
| MobileNet-V2          | ~300 MFLOPs            | 224×224    |
| ViT-Base (224×224/16) | ~17 GFLOPs             | 12L, D=768 |
| GPT-3 (175B)          | ~3e23 FLOPs / training | 一次训练       |
| LLaMA 7B inference    | ~80 GFLOPs/token       | 32 layers  |
